# Introduction to Tensorflow
**Tensorflow** is an open-source high-performance library for numerical computation that uses directed acyclic graphs (DAGs).

### Tensor
A **tensor** is an N-dimensional array of data.

![tensor.png](imgs/tensor.png)

A simple number like three or five is called a scaler. A vector is a one dimensional array of such numbers. Similarly, a two dimensional array is called a matrix and a three dimensional array is called a tensor. 

### Portability
TensorFlow graphs are portable between different devices. Portability between devices enables a lot of power and flexibility. For example, we can train a tensorflow model on the cloud that requires a lot of powerful hardware and then take that train model and put on a device out of the edge, perhaps a mobile phone or even embedded chip. This can help us in the predictions with the model right on that device itself. Due to this, Google translate can work completely offline because the trained translation model is stored on the phone and is available for offline translation. 

**TensorFlow Lite** provides on-device inference of ML models on mobile devices and is available for a variety of hardware.

### Tensorflow API Hierarchy
Tensorflow has a number of abstraction layers.
![tf_api.png](imgs/tf_api.png)

### Lazy Evaluation
The Python API lets us build and run Directed Graphs. For example, adding two tensors 'a' and 'b' returns a tensor 'c'.
```python
c=tf.add(a,b)
```
On running the above command, it doesn't execute it, rather it justs creates a DAG. In order to execute the DAG, we call a **session** to evaluate the command. So there are two steps:
* Create a Graph
* Run the graph

Thus, tensorflow is a **lazy evaluation** model.

NOTE: There is a separate mode called `tf.eager` which evaluates the operations immediately, and not lazily.

In [17]:
import tensorflow as tf
print(tf.__version__)

tf.compat.v1.disable_eager_execution() # need to disable eager mode

2.1.0


In [2]:
# Build the graph in a session

a = tf.constant([4,6,8], name='constant_a') # initializing a constant 
print(a)
b = tf.constant([1,2,3], name='constant_b')
print(b)

c = tf.add(a,b,name='constant_c') 
print(c) # returns a tensor

Tensor("constant_a:0", shape=(3,), dtype=int32)
Tensor("constant_b:0", shape=(3,), dtype=int32)
Tensor("constant_c:0", shape=(3,), dtype=int32)


In [3]:
# Sessions are used for the execution of TensorFlow graphs

with tf.compat.v1.Session() as sess: 
    print('Adding Tensors:',sess.run(c))

Adding Tensors: [ 5  8 11]


### Graphs and Sessions
![session.png](session.png)

In [4]:
with tf.compat.v1.Session() as sess: 

    a = tf.constant(5, name='a')
    b = tf.constant(2, name='b')
    c = tf.constant(3, name='c')
    d = tf.constant(4.0, name='d')
    e = tf.constant(16.0, name='e')
    
    mul = tf.multiply(a,b,name='mul')
    print('Multiplication: {}'.format(sess.run(mul)))
    
    div = tf.divide(d,e,name='div')
    print('\nDivision: {}'.format(sess.run(div)))
    
    try:
        add = mul+div # raises error since mul has type 'int' and div has type 'float'
    except TypeError as err:
        print('\nERROR: ',err)
        mul=tf.cast(mul,tf.float32) # Error Resolution
        add = mul+div
    print('Addition: {}'.format(sess.run(add)))
    
    power=tf.pow(b,c,name='power')
    print('\nPower: {}'.format(sess.run(power)))

    diff=d-e
    print('\nDifference: {}'.format(sess.run(diff)))

    sqrt=tf.sqrt(e,name='sqrt')
    print('\nSquare Root: {}'.format(sess.run(sqrt)))        

Multiplication: 10

Division: 0.25

ERROR:  Input 'y' of 'AddV2' Op has type float32 that does not match type int32 of argument 'x'.
Addition: 10.25

Power: 8

Difference: -12.0

Square Root: 4.0


### Slicing and Reshaping tensors

In [5]:
x = tf.constant([[4,6,8],[1,2,3]], name='constant_a')
print('Shape of Tensor x:',x.shape)
y = x[:,1] # Slicing tensor
z = tf.reshape(x,[3,2]) # reshaping tensor

with tf.compat.v1.Session() as sess:
    print('\nTensor y:',y.eval()) # .eval() is similar to .run()
    
    print('\nTensor z:')
    print (z.eval())

Shape of Tensor x: (2, 3)

Tensor y: [6 2]

Tensor z:
[[4 6]
 [8 1]
 [2 3]]


### Variables
A **variable** is a tensor whose value is initialized and then typically changes as the program runs.

In [11]:
def forward_pass(w,x):
    return tf.matmul(w,x)

# creating a variable tensor 'w', specifying how to initialize & whether it can be tuned
def train_loop(x,niter=5):
    with tf.compat.v1.variable_scope("model",reuse=tf.compat.v1.AUTO_REUSE):
        w=tf.compat.v1.get_variable("weights",shape=(1,2),initializer=tf.compat.v1.truncated_normal_initializer(),trainable=True)
    preds=[]
    for k in range(niter): # Trainable loop of 5 updates to weight
        preds.append(forward_pass(w,x))
        w=w+0.1
    return preds

with tf.compat.v1.Session() as sess:
    x=tf.constant([[3.2,6.7,2.1],[2.7,1.6,8.2]]) # 2X3 matrix
    preds=train_loop(x)
    tf.compat.v1.global_variables_initializer().run() # initalize all variables
    for i in range(len(preds)):
        print("{}:{}".format(i,preds[i].eval()))

0:[[ -6.6884823  -8.466469  -13.171674 ]]
1:[[ -6.0984817  -7.636469  -12.141673 ]]
2:[[ -5.508482   -6.8064694 -11.111673 ]]
3:[[ -4.918482   -5.9764695 -10.081673 ]]
4:[[-4.328482  -5.1464696 -9.051674 ]]


### Placeholder and feed_dict

**Placeholders** allows us to feed in values, such as by reading from a text file.

In [16]:
a = tf.compat.v1.placeholder("float",None)
b = a*4
c=a-b
print(a)
print(b)

with tf.compat.v1.Session() as sess:
    print(sess.run(c,feed_dict={a:[1,2,3]})) # passing values of 'a' using a feed_dict

Tensor("Placeholder_3:0", dtype=float32)
Tensor("mul_4:0", dtype=float32)
[-3. -6. -9.]
